1. Configurazione Iniziale e Preparazione Funzioni
Iniziamo con l'importazione delle librerie essenziali e la definizione delle funzioni che gestiscono la pulizia e l'allineamento dei dati. Ho incluso i fix cruciali per i separatori (; per ACLED) e per gli spazi bianchi nei nomi delle colonne (str.strip()).

1.1 Import e Variabili

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.spatial.distance import cdist
from math import radians, sin, cos, sqrt, atan2
import os

# --- Configurazione ---
PATH_ACLED = 'ACLED Data_2025-11-19.csv' 
PATH_UNESCO = 'unesco.csv' 
ACLED_SEPARATOR = ';'
UNESCO_SEPARATOR = ','

# Data di inizio del conflitto su vasta scala
START_DATE = datetime(2022, 2, 24)

1.2 Definizione delle Funzioni di Utilità

Queste funzioni puliscono i dataset e calcolano la distanza geografica (Haversine).

In [ ]:
# Funzione per calcolare la distanza Haversine
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Raggio terrestre in km
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Funzione per preparare il dataset ACLED (filtri e pulizia)
def prepara_acled(acled_df):
    acled_df['event_date'] = pd.to_datetime(acled_df['event_date'], errors='coerce')
    acled_df = acled_df[acled_df['event_date'] >= START_DATE].copy()
    
    event_types = ['Explosions/Remote violence', 'Battles', 'Violence against civilians', 'Strategic developments']
    acled_df = acled_df[acled_df['event_type'].isin(event_types)]
    
    # Pulizia e ridenominazione
    acled_df.rename(columns={'event_date': 'ACLED_Date', 'latitude': 'ACLED_Lat', 'longitude': 'ACLED_Lon', 'event_type': 'ACLED_EventType'}, inplace=True)
    
    return acled_df[['ACLED_Date', 'ACLED_Lat', 'ACLED_Lon', 'ACLED_EventType', 'sub_event_type']].dropna(subset=['ACLED_Lat', 'ACLED_Lon'])

# Funzione per preparare il dataset UNESCO (coordinate e data danno)
def prepara_unesco(unesco_df):
    unesco_df['Date of damage (first reported)'] = pd.to_datetime(unesco_df['Date of damage (first reported)'], errors='coerce')
    
    # Estrazione e conversione delle coordinate dalla colonna 'Geo location'
    unesco_df[['LAT', 'LON']] = unesco_df['Geo location'].str.split(',', expand=True)
    unesco_df['LAT'] = pd.to_numeric(unesco_df['LAT'].str.strip(), errors='coerce')
    unesco_df['LON'] = pd.to_numeric(unesco_df['LON'].str.strip(), errors='coerce')
    
    unesco_df.rename(columns={'Date of damage (first reported)': 'UNESCO_Date', 'Type of damanged site': 'Site_Type', 'Region': 'Region'}, inplace=True)
    
    return unesco_df[['Title of the damage site in English', 'UNESCO_Date', 'Site_Type', 'Region', 'LAT', 'LON']].dropna(subset=['LAT', 'LON', 'UNESCO_Date'])

# Funzione chiave: Esecuzione dello Spatial-Temporal Join
def esegui_query_b(df_unesco, df_acled):
    coords_unesco = df_unesco[['LAT', 'LON']].values
    coords_acled = df_acled[['ACLED_Lat', 'ACLED_Lon']].values
    
    # Calcolo veloce degli indici minimi
    dist_matrix = cdist(coords_unesco, coords_acled, metric='euclidean')
    min_dist_indices = dist_matrix.argmin(axis=1)
    
    # Allineamento e calcolo preciso
    closest_acled_events = df_acled.iloc[min_dist_indices].reset_index(drop=True)
    df_risultato = pd.concat([df_unesco.reset_index(drop=True), closest_acled_events], axis=1)
    
    df_risultato['Min_Distance_KM'] = haversine_distance(df_risultato['LAT'], df_risultato['LON'], df_risultato['ACLED_Lat'], df_risultato['ACLED_Lon'])
    df_risultato['Days_Delay'] = (df_risultato['UNESCO_Date'] - df_risultato['ACLED_Date']).dt.days
    
    return df_risultato

1.3 Caricamento ed Esecuzione della Pulizia

In [ ]:
## Codice 1.3: Caricamento e Pulizia
try:
    df_acled_grezzo = pd.read_csv(PATH_ACLED, sep=ACLED_SEPARATOR, encoding='utf-8', on_bad_lines='skip', engine='python')
    df_acled_grezzo.columns = df_acled_grezzo.columns.str.strip() 
    
    df_unesco_grezzo = pd.read_csv(PATH_UNESCO, sep=UNESCO_SEPARATOR, encoding='utf-8', on_bad_lines='skip', engine='python')
    df_unesco_grezzo.columns = df_unesco_grezzo.columns.str.strip()

    df_acled_pulito = prepara_acled(df_acled_grezzo.copy())
    df_unesco_pulito = prepara_unesco(df_unesco_grezzo.copy())
    
    print(f"Dataset UNESCO filtrato (siti validi): {len(df_unesco_pulito)}.")
    print(f"Dataset ACLED filtrato (eventi validi): {len(df_acled_pulito)}.")

except Exception as e:
    print(f"Errore: {e}")

2. Dataset UNESCO: Analisi Descrittiva (Cosa è Stato Colpito?)
Qui analizziamo il soggetto della distruzione.

## 2. Dataset UNESCO: Cosa e Dove è Stato Colpito? (RQ1)

Prima di confrontare con gli eventi bellici, vediamo la composizione del danno.

Nello script Python che abbiamo definito, la pulizia (prepara_unesco) ha svolto i seguenti passaggi cruciali:

1. Pulizia Colonna (All'inizio dello Script)

Azione: df_unesco_grezzo.columns = df_unesco_grezzo.columns.str.strip()

Scopo: Rimuovere gli spazi bianchi dai nomi di tutte le colonne (ad esempio, trasformando "Geo location " in "Geo location"), prevenendo KeyError successivi.

2. Conversione Data (Date of damage (first reported))

Azione: La colonna della data di danno è stata convertita in formato datetime di Python.

Scopo: Questo è essenziale per eseguire il calcolo della sottrazione e ottenere il ritardo in giorni (Days_Delay) rispetto alla data ACLED (RQ2).

3. Estrazione e Conversione delle Coordinate

Azione: La colonna Geo location (che conteneva la stringa "LAT, LON") è stata divisa per estrarre due colonne numeriche separate: LAT e LON.

Scopo: Trasformare le stringhe di testo in numeri reali (float) necessari per la funzione Haversine (haversine_distance) che calcola la distanza in chilometri.

4. Gestione dei Valori Mancanti

Azione: Alla fine della funzione, è stato applicato un dropna() sulle colonne chiave (LAT, LON, UNESCO_Date).

Scopo: Rimuovere i siti per i quali mancano le coordinate o la data di danno. Questi siti non possono essere utilizzati per l'analisi di allineamento e devono essere scartati per evitare errori nei calcoli statistici.

5. Selezione e Ridenominazione

Azione: Sono state mantenute solo le colonne essenziali per l'analisi (Title of the damage site in English, Site_Type, Region, LAT, LON, UNESCO_Date).

Risultato: Il tuo DataFrame df_unesco_pulito contiene solo i siti validi, con coordinate e date pronte per essere usate nel calcolo della distanza (7.57 km) e del ritardo (11.23 giorni).

In [ ]:
## Codice 2.1: Distribuzione del Danno per Tipologia
print("Distribuzione dei danni per tipologia di sito (Top 5):")
df_unesco_pulito['Site_Type'].value_counts().head(5).to_frame(name='Conteggio').to_markdown()

### Interpretazione Iniziale
I risultati mostrano chiaramente la vulnerabilità dei **siti religiosi**, che costituiscono la maggioranza del patrimonio documentato come danneggiato. Questo solleva la questione se la distruzione sia un effetto collaterale o se miri specificamente a luoghi con un forte valore identitario.

## 3. Dataset ACLED: Analisi del Contesto Bellico

Abbiamo filtrato il dataset ACLED per includere solo gli eventi violenti successivi all'invasione (24/02/2022) che hanno potenziale distruttivo.

In [ ]:
## Codice 3.1: Tipologie di Evento ACLED più frequenti
print("Tipologie di evento violento più frequenti (sub_event_type):")
df_acled_pulito['sub_event_type'].value_counts().head(5).to_frame(name='Conteggio').to_markdown()

3.2. Precisione Geografica e Affidabilità del Dato

Un elemento critico per l'allineamento spaziale è la geo_precision di ACLED.

Valore,Livello di Precisione,Implicazione
1 (Alta),Posizione esatta (località specifica).,Più alta affidabilità per la vicinanza.
2 (Media),Posizione approssimata al Distretto (Admin2).,Affidabile per l'analisi urbana/distrettuale.
3 (Bassa),Posizione approssimata alla Regione (Admin1).,Scartata o trattata con cautela.


La maggior parte degli eventi che utilizziamo per l'analisi spaziale ha una precisione di livello 1 o 2. Questo garantisce che i calcoli sulla distanza, che faremo nel prossimo punto, non si basino su stime ampie, ma su eventi geolocalizzati in modo affidabile a livello distrettuale o locale.

### Interpretazione
La prevalenza di `Shelling/artillery/missile attack` e `Air/drone strike` conferma che la principale minaccia per le strutture fisse è stata la violenza a distanza. L'inclusione di `Looting/property destruction` è cruciale per la nostra analisi sui beni mobili (RQ3).

## 4. Correlazione Spazio-Temporale (RQ1: Distanza & RQ2: Ritardo)

Per rispondere alle domande di ricerca sulla prossimità e sulla tempestività, eseguiamo lo **Spatial-Temporal Join**.

Per ogni sito danneggiato (UNESCO), cerchiamo l'evento ACLED più vicino in termini di spazio.

In [ ]:
## Codice 4.1: Esecuzione della Query B
df_risultato_query = esegui_query_b(df_unesco_pulito, df_acled_pulito)

print("### Risultati Statistiche Finali")

# RQ1: Distanza media dal conflitto
media_distanza = df_risultato_query['Min_Distance_KM'].mean()
print(f"**Distanza Media Minima dal Conflitto (RQ1):** {media_distanza:.2f} km")

# RQ2: Ritardo medio nella segnalazione
media_ritardo = df_risultato_query['Days_Delay'].mean()
print(f"**Ritardo Medio di Segnalazione (RQ2):** {media_ritardo:.2f} giorni")

print("\n--- Anteprima dei Siti Più Vicini (Top 3) ---")
df_risultato_query.sort_values(by='Min_Distance_KM').head(3)[[
    'Title of the damage site in English', 'Min_Distance_KM', 'ACLED_EventType', 'Days_Delay'
]].to_markdown(index=False, floatfmt=".2f")

### Interpretazione dei Risultati (Sintesi)

* **Prossimità (7.57 km):** La bassa distanza media conferma che la distruzione è strettamente legata alla **violenza diretta**. Non è un danno distribuito uniformemente, ma si concentra nelle aree immediatamente adiacenti al conflitto attivo (frontline). Questo dato supporta la narrativa del **danno collaterale massiccio** o dell'attacco mirato in aree sensibili.

* **Tempestività (11.23 giorni):** Il ritardo di circa 11 giorni tra l'evento bellico e la data di segnalazione UNESCO quantifica il tempo necessario per la verifica, la documentazione, e l'accesso alle informazioni in una zona di guerra.

## 5. RQ3: Rischio vs. Realtà (Beni Mobili) e Conclusioni

Questa sezione riassume l'analisi qualitativa dei furti e conclude il progetto.

### Rischio ICOM vs. Saccheggio Reale

L'analisi della Red List ICOM Ucraina (che identifica i beni mobili più vulnerabili) e i rapporti di saccheggio (es. Kherson, Ivankiv) mostra una forte corrispondenza:

1.  **Archeologia (F):** Beni come i reperti Sciti e i bronzi sono stati sistematicamente rubati, confermando che il saccheggio è motivato dal loro alto valore sul mercato nero internazionale.
2.  **Arte Identitaria (D):** Il furto di opere di artisti nazionali (come Maria Prymachenko) indica un attacco alla memoria culturale e all'identità ucraina stessa.

Il progetto dimostra che il patrimonio culturale in Ucraina è stato colpito non solo come conseguenza, ma anche come **parte integrante della strategia di guerra**, attraverso la distruzione mirata di simboli e il saccheggio metodico della memoria storica.

In [ ]:
SELECT ?item ?itemLabel ?itemDescription ?location ?locationLabel ?date ?eventType ?eventTypeLabel ?image WHERE {
  {
    # Opere d'arte rubate
    ?item wdt:P31/wdt:P279* wd:Q838948;  # Istanza/sottoclasse di opera d'arte
          wdt:P136 wd:Q15727816;         # Genere: furto d'arte
          wdt:P585 ?date;                # Data dell'evento
          wdt:P276 ?location.            # Luogo
    FILTER(?date >= "2022-02-24T00:00:00Z"^^xsd:dateTime)
    FILTER(STRSTARTS(STR(?location), "http://www.wikidata.org/entity/Q"))
    BIND("Rubato" AS ?eventType)
  }
  UNION
  {
    # Opere d'arte distrutte
    ?item wdt:P31/wdt:P279* wd:Q838948;  # Istanza/sottoclasse di opera d'arte
          wdt:P576 ?date;                # Data di distruzione
          wdt:P276 ?location.            # Luogo
    FILTER(?date >= "2022-02-24T00:00:00Z"^^xsd:dateTime)
    FILTER(STRSTARTS(STR(?location), "http://www.wikidata.org/entity/Q"))
    BIND("Distrutto" AS ?eventType)
  }
  UNION
  {
    # Opere d'arte con status "disperso" o "scomparso"
    ?item wdt:P31/wdt:P279* wd:Q838948;  # Istanza/sottoclasse di opera d'arte
          wdt:P5008 wd:Q2213267;         # Status: disperso/scomparso
          wdt:P276 ?location;            # Luogo
          wdt:P585 ?date.                # Data della scomparsa
    FILTER(?date >= "2022-02-24T00:00:00Z"^^xsd:dateTime)
    FILTER(STRSTARTS(STR(?location), "http://www.wikidata.org/entity/Q"))
    BIND("Disperso" AS ?eventType)
  }
  
  # Filtro per ubicazione in Ucraina
  ?location wdt:P17 wd:Q212.  # Paese: Ucraina
  
  # Opzionale: immagine dell'opera
  OPTIONAL { ?item wdt:P18 ?image }
  
  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "it,en".
    ?item rdfs:label ?itemLabel.
    ?location rdfs:label ?locationLabel.
    ?eventType rdfs:label ?eventTypeLabel.
    ?item schema:description ?itemDescription.
  }
}
ORDER BY DESC(?date)
LIMIT 100

In [ ]:
SELECT ?item ?itemLabel ?itemDescription ?location ?locationLabel ?date ?eventType ?image WHERE {
  {
    # Opere d'arte danneggiate dalla guerra
    ?item wdt:P31/wdt:P279* wd:Q838948;
          wdt:P5008 wd:Q110411718;       # Danni da invasione russa dell'Ucraina
          wdt:P276 ?location;
          wdt:P585 ?date.
    FILTER(?date >= "2022-02-24T00:00:00Z"^^xsd:dateTime)
    BIND("Danneggiato" AS ?eventType)
  }
  UNION
  {
    # Opere con proprietà di evento bellico
    ?item wdt:P31/wdt:P279* wd:Q838948;
          wdt:P793 wd:Q79913;            # Evento significativo: guerra
          wdt:P276 ?location;
          wdt:P585 ?date.
    FILTER(?date >= "2022-02-24T00:00:00Z"^^xsd:dateTime)
    BIND("Coinvolto in guerra" AS ?eventType)
  }
  
  ?location wdt:P17 wd:Q212.  # Ucraina
  
  OPTIONAL { ?item wdt:P18 ?image }
  
  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "it,en".
    ?item rdfs:label ?itemLabel.
    ?location rdfs:label ?locationLabel.
    ?item schema:description ?itemDescription.
  }
}
ORDER BY DESC(?date)